<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250616%5D%20TrustFed-IDS%20%E2%80%93%20WSN-BFSF%20%E2%80%93%2010%20%25%20sign-flip%20attack%20(client%202).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TrustFed-IDS – WSN-BFSF – 10 % sign-flip attack (client 2)**

In [1]:
#!/usr/bin/env python3
# TrustFed-IDS – WSN-BFSF – 10 % sign-flip attack (client 2)
import os, time, psutil, numpy as np, pandas as pd, tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers.schedules import CosineDecay
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.spatial.distance import cosine

In [2]:
# ---------- global config ----------
SEED, NUM_CLIENTS, ATTACKER_ID = 42, 5, 2     # client-id 2 is malicious
ROUNDS, LOCAL_EPOCHS           = 75, 1
BATCH_SIZE, DIRICHLET_ALPHA    = 32, 0.5
HISTORY_KEEP                   = 6
TRUST_ALPHA, TRUST_CLIP        = (0.30,0.55,0.15), (0.05,0.60)
LOG_DIR, DATA_PATH             = "/mnt/data", "dataset_WSN-BFSF.csv"

np.random.seed(SEED); tf.random.set_seed(SEED)

In [3]:
# ---------- data loading ----------
df = pd.read_csv("dataset.csv").dropna()
for col in df.select_dtypes(include="object"):
    df[col] = LabelEncoder().fit_transform(df[col])
X_all = df.drop("Class", axis=1).astype("float32").values
y_all = df["Class"].astype("int64").values

X_tr, X_te, y_tr, y_te = train_test_split(
    X_all, y_all, test_size=0.20, stratify=y_all, random_state=SEED)
scaler = StandardScaler().fit(X_tr)
X_tr, X_te = scaler.transform(X_tr), scaler.transform(X_te)
X_tr, y_tr = SMOTE(random_state=SEED).fit_resample(X_tr, y_tr)

X_tr, X_te = X_tr[..., None], X_te[..., None]
num_classes = int(y_tr.max()+1)
y_te_cat    = to_categorical(y_te, num_classes)

In [4]:
# ---------- client partition ----------
def dirichlet_split(X, y, k, alpha, rng):
    idx = {c: np.where(y==c)[0] for c in np.unique(y)}
    cl  = [[] for _ in range(k)]
    for c, ids in idx.items():
        rng.shuffle(ids)
        parts = (rng.dirichlet([alpha]*k)*len(ids)).astype(int)
        while parts.sum()<len(ids): parts[rng.randint(0,k)]+=1
        s=0
        for cid, ct in enumerate(parts):
            cl[cid].extend(ids[s:s+ct]); s+=ct
    for lst in cl: rng.shuffle(lst)
    return [X[l] for l in cl], [y[l] for l in cl]

rng = np.random.RandomState(SEED)
client_X_raw, client_y_raw = dirichlet_split(X_tr, y_tr, NUM_CLIENTS,
                                             DIRICHLET_ALPHA, rng)
client_X = client_X_raw
client_y = [to_categorical(y, num_classes) for y in client_y_raw]
INPUT_SHAPE = (X_tr.shape[1],1)

In [5]:
# ---------- model ----------
def build_model(inp=INPUT_SHAPE, classes=num_classes):
    sched = CosineDecay(5e-4, decay_steps=ROUNDS, alpha=0.4)
    opt   = tf.keras.optimizers.Nadam(sched, clipnorm=2.0)
    m = Sequential([
        InputLayer(input_shape=inp),
        LSTM(128, return_sequences=True, activation='tanh',
             kernel_regularizer=l2(5e-4)),
        LSTM(64, activation='tanh', kernel_regularizer=l2(5e-4)),
        Dense(256, activation='relu'), Dropout(0.20),
        Dense(128, activation='relu'), Dropout(0.25),
        Dense(classes, activation='softmax')
    ])
    focal = tf.keras.losses.CategoricalFocalCrossentropy(
                alpha=0.5, gamma=2.0)
    m.compile(opt, loss=focal, metrics=['accuracy'])
    return m

# ---------- helpers ----------
def weight_update(lw, gw): return [l-g for l,g in zip(lw,gw)]
def vec_cos(a,b):
    v1,v2 = np.concatenate([w.ravel() for w in a]), np.concatenate([w.ravel() for w in b])
    return 0.0 if (np.all(v1==0) or np.all(v2==0) ) else 1-cosine(v1,v2)
def stability(u,h): return 1.0 if len(h)<2 else float(np.nanmean([vec_cos(u,x) for x in h[-HISTORY_KEEP:]]))
def compute_trust(upd,vloss,hist):
    lo,hi = min(vloss.values()), max(vloss.values())
    raw = {cid:(TRUST_ALPHA[0]*vec_cos(u,[np.zeros_like(w) for w in u]) +
                TRUST_ALPHA[1]*(1-(vloss[cid]-lo)/(hi-lo+1e-8)) +
                TRUST_ALPHA[2]*stability(u,hist[cid]))
           for cid,u in upd.items()}
    clipped = {cid:np.clip(s,*TRUST_CLIP) for cid,s in raw.items()}
    Z = sum(clipped.values())
    return {cid:s/Z for cid,s in clipped.items()}

def aggregate(W,T,N):
    tot = sum(T[c]*N[c] for c in W)
    return [sum(T[c]*N[c]*W[c][l] for c in W)/tot
            for l in range(len(next(iter(W.values()))))]

# ---------- training ----------
g_model = build_model(); g_weights = g_model.get_weights()
histories = {c:[] for c in range(NUM_CLIENTS)}
perf_log, comm_log, trust_log = [], [], []
cls_wt = dict(enumerate(compute_class_weight('balanced',
                                             classes=np.arange(num_classes),
                                             y=y_tr)))

for rnd in range(1, ROUNDS+1):
    t0=time.time(); lw,upd,vloss,ns,bytes_out={}, {}, {}, {}, 0
    for cid in range(NUM_CLIENTS):
        n_val = max(1,int(0.15*len(client_X[cid])))
        Xv,yv = client_X[cid][:n_val], client_y[cid][:n_val]
        Xt,yt = client_X[cid][n_val:], client_y[cid][n_val:]

        local = build_model(); local.set_weights(g_weights)
        local.fit(Xt,yt,epochs=LOCAL_EPOCHS,batch_size=BATCH_SIZE,
                  verbose=0,class_weight=cls_wt)

        w = local.get_weights()
        u = weight_update(w, g_weights)

        # ---- gradient sign-flip attack ----
        if cid == ATTACKER_ID:
            u = [-layer for layer in u]

        l = local.evaluate(Xv,yv,verbose=0)[0]
        lw[cid],upd[cid],vloss[cid],ns[cid] = w,u,l,len(Xt)
        histories[cid] = (histories[cid]+[u])[-HISTORY_KEEP:]
        bytes_out += sum(x.nbytes for x in w)

    trust = compute_trust(upd, vloss, histories)
    g_weights = aggregate(lw, trust, ns); g_model.set_weights(g_weights)

    y_pred = np.argmax(g_model.predict(X_te,verbose=0),axis=1)
    perf_log.append(dict(round=rnd,
                         accuracy=accuracy_score(y_te,y_pred),
                         precision=precision_score(y_te,y_pred,average='weighted',zero_division=0),
                         recall=recall_score(y_te,y_pred,average='weighted',zero_division=0),
                         f1=f1_score(y_te,y_pred,average='weighted',zero_division=0),
                         ms=round((time.time()-t0)*1000,2)))
    comm_log.append({"round":rnd,"MB":bytes_out/2**20})
    trust_log.extend([{"round":rnd,"client":c,"trust":t,
                       "malicious":(c==ATTACKER_ID)}
                      for c,t in trust.items()])

    print(f"R{rnd:02d}  acc={perf_log[-1]['accuracy']:.8f} "
          f"F1={perf_log[-1]['f1']:.8f}  MB={bytes_out/2**20:.8f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


R01  acc=0.21255327 F1=0.21773727  MB=3.16658020


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


R02  acc=0.53056615 F1=0.63127181  MB=3.16658020


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...

R73  acc=0.87022204 F1=0.89859393  MB=3.16658020


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


R74  acc=0.86285284 F1=0.89337148  MB=3.16658020


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


R75  acc=0.85144661 F1=0.88531472  MB=3.16658020


In [6]:
# ---------- save logs ----------
os.makedirs(LOG_DIR,exist_ok=True)
pd.DataFrame(perf_log ).to_csv(f"{LOG_DIR}/perf_log_BFSF_trust_poison.csv", index=False)
pd.DataFrame(comm_log ).to_csv(f"{LOG_DIR}/comm_log_BFSF_trust_poison.csv", index=False)
pd.DataFrame(trust_log).to_csv(f"{LOG_DIR}/trust_log_BFSF_trust_poison.csv", index=False)
print("\n✓ TrustFed-IDS poisoned run complete")


✓ TrustFed-IDS poisoned run complete


In [7]:
# prompt: Download all logs files aboved
from google.colab import files
files.download(f"{LOG_DIR}/perf_log_BFSF_trust_poison.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
files.download(f"{LOG_DIR}/comm_log_BFSF_trust_poison.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
files.download(f"{LOG_DIR}/trust_log_BFSF_trust_poison.csv")
#files.download(f"{LOG_DIR}/model_profile_BFSF_trustcap60.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
# prompt: Download all logs files aboved

from google.colab import files
import os

# Define the directory where the log files are saved
log_directory = LOG_DIR

# Get a list of all files in the log directory
log_files = [f for f in os.listdir(log_directory) if os.path.isfile(os.path.join(log_directory, f))]

# Download each log file
for log_file in log_files:
  try:
    files.download(os.path.join(log_directory, log_file))
    print(f"Downloaded: {log_file}")
  except Exception as e:
    print(f"Error downloading {log_file}: {e}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: perf_log_BFSF_trust_poison.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: trust_log_BFSF_trust_poison.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: comm_log_BFSF_trust_poison.csv
